In [7]:
%reload_ext autoreload
%autoreload 2

iter = 9

import numpy as np; # np.set_printoptions(precision = 8)
import gmsh
import pde
import scipy.sparse as sps
import scipy.sparse.linalg
import time
import sys
print(sys.path)

#import sys
#!{sys.executable} -m pip install scikit-sparse

from sksparse.cholmod import cholesky
# from scipy.sparse.linalg import inv as sinv
# from scikits.umfpack import spsolve, splu

pex = lambda x,y : (1-x)**4+(1-y)**3*(1-x) + np.sin(1-y)*np.cos(1-x)
u1ex = lambda x,y : -np.sin(1-x)*np.sin(1-y)-4*(x-1)**3-(y-1)**3
u2ex = lambda x,y : -3*(x-1)*(y-1)**2 + np.cos(1-x)*np.cos(1-y)
divuex = lambda x,y : -6*(x-1)*(2*x+y-3)+2*np.cos(1-x)*np.sin(1-y)

# pex = lambda x,y : x+y
# u1ex = lambda x,y : -1+0*x
# u2ex = lambda x,y : -1+0*x
# divuex = lambda x,y : 0+0*x

['/Users/catalinradu/Documents/matlab/wild_python', '/Users/catalinradu/opt/anaconda3/lib/python39.zip', '/Users/catalinradu/opt/anaconda3/lib/python3.9', '/Users/catalinradu/opt/anaconda3/lib/python3.9/lib-dynload', '', '/Users/catalinradu/.local/lib/python3.9/site-packages', '/Users/catalinradu/opt/anaconda3/lib/python3.9/site-packages', '/Users/catalinradu/opt/anaconda3/lib/python3.9/site-packages/aeosa']


In [ ]:
error_uh = np.empty(iter)
error_ph = np.empty(iter)

geo = 'mesh_new.geo'
# geo = 'unit_square.geo'

for i in range(iter):
    
    print("Iteration",str(i),"/",str(iter-1))
    p,e,t,q = pde.petq_from_gmsh(filename = geo,hmax = 1*1/np.sqrt(2)**i)
    
    MESH = pde.initmesh(p,e,t,q); BASIS = pde.basis(); LISTS = pde.lists(MESH)
    
    MAT = {}
    MAT = MAT | pde.assemble.hdiv(MESH,BASIS,LISTS,space = 'BDM1-BDM1')
    MAT = MAT | pde.assemble.h1(MESH,BASIS,LISTS,space = 'P1d-Q1d')
    
    p_n_BDM1  = pde.projections.assem_HDIV_b(MESH, BASIS, dict(space = 'BDM1', edges = MESH.Boundary.TrigEdges, order = 2), pex)
    p_n_BDM1 += pde.projections.assem_HDIV_b(MESH, BASIS, dict(space = 'BDM1', edges = MESH.Boundary.QuadEdges, order = 0), pex)

    M_divuex_P0_Q0_new = pde.projections.assem(MESH, BASIS, LISTS, dict(trig = 'P0',quad = 'Q0'), divuex)
    
    M = MAT['BDM1-BDM1']['M']
    C = MAT['BDM1-BDM1']['C']
    D = MAT['BDM1-BDM1']['D']
    Mh = MAT['BDM1-BDM1']['Mh']
    Mx_P1d_Q1d = MAT['BDM1-BDM1']['Mx_P1d_Q1d']
    My_P1d_Q1d = MAT['BDM1-BDM1']['My_P1d_Q1d']
    M_P1d_Q1d = MAT['P1d-Q1d']['M']
    
    Z = sps.coo_matrix((C.shape[0],C.shape[0]), dtype = np.float64)
    
    A = sps.vstack((sps.hstack((Mh,-C.T)),
                    sps.hstack((C,   Z))))
    
#    b = np.r_[-p_n_BDM1,
#               M_divuex_P0_Q0_new]
    
    cholMh = cholesky(Mh)

    Afun = lambda x : (C@(cholMh.solve_A(C.T@x)))
    b = M_divuex_P0_Q0_new - C@(cholMh.solve_A(-p_n_BDM1))
    ph = pde.pcg(Afun,b,maxit=1e10,tol=1e-15)
    uh = cholMh.solve_A(C.T@ph-p_n_BDM1)
    
    
#    res = sps.linalg.spsolve(A,b)
    
#   uh = res[:2*MESH.NoEdges]; ph = res[2*MESH.NoEdges:]
    
    uex_BDM1 = pde.projections.interp_HDIV(MESH, BASIS, 'BDM1', lambda x,y : np.c_[u1ex(x,y),u2ex(x,y)])
    pex_P0 = pde.projections.evaluate(MESH, dict(trig = 'P0',quad = 'Q0'), pex)
    
#     uhr = uh.reshape((uh.size//2,2))
#     uhr = np.mean(uhr,1)
#     uhr = np.tile(uhr, (2,1)).flatten('F')
#     uh_old = uh
#     uh = uhr
    
    
    error_uh[i] = np.sqrt((uh-uex_BDM1)@M@(uh-uex_BDM1))
    error_ph[i] = np.sqrt((ph-pex_P0)@D@(ph-pex_P0))

Iteration 0 / 8
Generated mesh with 12 points, 10 boundary edges, 8 triangles, 2 quadrilaterals.
Assembling BDM1-BDM1 took 0.011 seconds.
Assembling P1d-Q1d took 0.003 seconds.
pcg stopped after 9 iterations with relres 3.0261706387988335e-20
Iteration 1 / 8
Generated mesh with 55 points, 26 boundary edges, 46 triangles, 18 quadrilaterals.
Assembling BDM1-BDM1 took 0.011 seconds.
Assembling P1d-Q1d took 0.003 seconds.
pcg stopped after 39 iterations with relres 4.341034895613181e-16
Iteration 2 / 8
Generated mesh with 205 points, 58 boundary edges, 154 triangles, 98 quadrilaterals.
Assembling BDM1-BDM1 took 0.015 seconds.
Assembling P1d-Q1d took 0.003 seconds.
pcg stopped after 79 iterations with relres 9.309609719744065e-16
Iteration 3 / 8
Generated mesh with 887 points, 128 boundary edges, 620 triangles, 512 quadrilaterals.
Assembling BDM1-BDM1 took 0.024 seconds.
Assembling P1d-Q1d took 0.005 seconds.
pcg stopped after 171 iterations with relres 8.260980643020891e-16
Iteration 4 / 8

In [ ]:
rate = np.log2(error_uh[0:-1]/error_uh[1:])
print(rate)

rate = np.log2(error_ph[0:-1]/error_ph[1:])
print(rate)

In [9]:
print(error_ph)

[1.19840830e+01 1.66402479e+00 5.94941851e-01 1.43162373e-01 3.10389497e-02 6.09392303e-03 1.39083139e-03 7.96006155e-04 4.95651714e-04]


In [10]:
fig = MESH.pdesurf_hybrid(dict(trig = 'P0',quad = 'Q0'),ph)
fig.show()

In [20]:
ph[:10]

array([ 22.36968894,  22.45951225,  29.90747597,  29.5862177 , 149.16690598,  28.05482116,  45.23770179,  45.03284317,  34.84669655, 148.51707344])

In [21]:
pex_P0[:10]

array([ 22.37040839,  22.46008735,  29.90824087,  29.58677939, 149.1695595 ,  28.05645398,  45.23931148,  45.03463485,  34.84790278, 148.51926478])

In [4]:
iMh.shape

(332220, 103951)

In [18]:
import sys
print(sys.path)

['/Users/catalinradu/Documents/matlab/wild_python', '', '/Users/catalinradu/.local/lib/python3.9/site-packages', '/opt/intel/oneapi/intelpython/latest/lib/python3.9/site-packages']


In [4]:
sys.path.remove("/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages");

In [6]:
sys.path.remove("/Users/catalinradu/Library/Python/3.11/lib/python/site-packages"); 

In [9]:
/opt/intel/oneapi/intelpython/latest/lib/python3.9/site-packages

SyntaxError: invalid syntax (4073746558.py, line 1)

In [10]:
sys.path.append("/opt/intel/oneapi/intelpython/latest/lib/python3.9/site-packages")

In [17]:
sys.path.remove("/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/lib-dynload")

In [20]:
!python

zsh:1: command not found: python


In [21]:
-version

NameError: name 'version' is not defined

In [6]:
from platform import python_version

print(python_version())

3.9.13
